Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE". You can run all the tests with the validate button. If the validate command takes too long, you can also confirm that you pass all the tests if you can run through the whole notebook without getting validation errors.

For this problem set, we'll be using the Jupyter notebook:

![](jupyter.png)

## Music recommender exercises

In this notebook you will make recommendations based on user's music preferences. Training is done with alternating least squares (ALS) recommendation model to recommend songs to users.

https://spark.apache.org/docs/latest/mllib-collaborative-filtering.html#collaborative-filtering

https://spark.apache.org/docs/latest/api/python/_modules/pyspark/mllib/recommendation.html

dataset by:  
Thierry Bertin-Mahieux, Daniel P.W. Ellis, Brian Whitman, and Paul Lamere.   
The Million Song Dataset. In Proceedings of the 12th International Society  
for Music Information Retrieval Conference (ISMIR 2011), 2011.  

We use a sample data of "uniqe_tracks.txt" from http://millionsongdataset.com/sites/default/files/AdditionalFiles/unique_tracks.txt.

In [ ]:
import pyspark.mllib
from pyspark.sql import *
from pyspark import *
from pyspark.rdd import *
from pyspark.ml import *
from pyspark.ml.feature import *
from pyspark.ml.classification import *
from pyspark.sql.types import *
from pyspark.mllib.recommendation import *
import random


sc = SparkContext("local","music")
spark = SparkSession(sc)

sampleUsersPath = "sampleUsers.txt"
sampleTracksPath = "sampleTracks.txt"

## Load

Load data from path and create a dataframe from it. It should have three (3) columns:  
-> `user`: String (user_id)  
-> `song`: String (song_id)  
-> `count`: Int (number of times listened)  

Columns are seperated by "\t".  
Counts that are higher than 20 should be reduced to 20.  
Using the `StructField`schema can help.

param `path` path to file (users.txt)  
`return` DataFrame

In [ ]:
def load(path):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
loaded = load(sampleUsersPath).persist()
loaded.show()

In [ ]:
'''load test'''
correctCols = StructType([\
StructField("user",StringType(),True),\
StructField("song",StringType(),True),\
StructField("count",IntegerType(),True)])

fakeData = [("abc123", "123abc", 2)]

fakeDf = spark.createDataFrame(fakeData, correctCols)

assert loaded.dtypes == fakeDf.dtypes, "the schema was expected to be %s but it was %s" % (fakeDf.dtypes, loaded.dtypes)

assert loaded.filter('count>20').count() == 0, "counts higher than 20 was expected to be 0 but it was %s" % loaded.filter('count>20').count()

test1 = str(loaded.sample(False, 0.01, seed=123).limit(1).first())
correct1 = "Row(user='a58de017cbeda1763ea002fe027ed41b4ed53109', song='SOJTLHS12A8C13F633', count=3)"
assert test1 == correct1, "the row was expected to be %s but it was %s" % (correct1, test1)


## Convert

Convert user and song fields into doubles. Use mllib's StringIndexer. Final schema:  
-> `user`: String  
-> `song`: String  
-> `count`: Int  
-> `user_indexed`: Double  
-> `song_indexed`: Double  

param `df` Dataframe which has been created using method `load()`  
`return` Dataframe


In [ ]:
def convert(df):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
converted = convert(loaded).persist()
converted.show()

In [ ]:
'''convert test'''
correctCols = StructType([\
StructField("user",StringType(),True),\
StructField("song",StringType(),True),\
StructField("count",IntegerType(),True),\
StructField("user_indexed",DoubleType(),True),\
StructField("song_indexed",DoubleType(),True)])

fakeData = [("abc123", "123abc", 2, 1.0, 2.0)]

fakeDf = spark.createDataFrame(fakeData, correctCols)

assert converted.dtypes == fakeDf.dtypes, "the schema was expected to be %s but it was %s" % (fakeDf.dtypes, converted.dtypes)

test2 = str(converted.sample(False, 0.1, seed=1234).limit(1).first())
correct2 = "Row(user='5a905f000fc1ff3df7ca807d57edb608863db05d', song='SOCHPFL12AF72A3F64', count=2, user_indexed=5.0, song_indexed=767.0)"
assert test2 == correct2, "the row was expected to be %s but it was %s" % (correct2, test2)


## To Rating

create RDD of Rating classes. Note that you need to use  
https://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#pyspark.mllib.recommendation.Rating

The params of the Rating function should be user=`user_indexed`, product=`song_indexed`and rating=`count`

param `df` Dataframe which has `user_indexed` and `song_indexed` fields (output from `convert()` method)  
`return` RDD containg Rating objects


In [ ]:
def toRating(df):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
rated = toRating(converted).persist()
rated.take(10)

In [ ]:
'''toRating tests'''
rows = [Rating(user=162, product=577, rating=2.0),
 Rating(user=162, product=1053, rating=1.0),
 Rating(user=162, product=1646, rating=1.0),
 Rating(user=162, product=1945, rating=1.0),
 Rating(user=162, product=2306, rating=1.0)]
assert rated.take(5) == rows, "the first 5 rows were expected to be %s but they were %s" % (rows, rated.take(5))

random.seed(54321)
r = random.randint(100, 2000)

test3 = str(toRating(converted).collect()[r])
correct3 = "Rating(user=599, product=1321, rating=1.0)"
assert test3 == correct3, "the row was expected to be %s but it was %s" % (correct3, test3)


## Train ALS

train ALS model. For testing purposes you have to include a seed, e.g seed=seed.

https://spark.apache.org/docs/latest/mllib-collaborative-filtering.html#collaborative-filtering

param `data` RDD of Rating objects that is created using `toRating()` method.  
param `seed` value used for testing purposes. 
`return` trained ALS model

In [ ]:
def trainALS(data, seed):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
random.seed(123)
rSeed = random.randint(0, 10000)
model = trainALS(rated, rSeed)

## Recommend Songs

Recommend 5 songs to a given user.

param `model` trained ALS model  
param `user` user id converted (user_indexed) to Integer (with `convert()`)  
`return` recommendations in Array  


In [ ]:
def recommendSongs(model, user):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
recommends = recommendSongs(model, 162)
recommends

In [ ]:
'''model and recommendSongs tests'''
assert type(recommends[0]) == pyspark.mllib.recommendation.Rating, "the type was expected to be pyspark.mllib.recommendation.Rating  but it was %s" % type(recommends[0]) 
assert recommends[0].user == 162, "the user was expected to be 162 but it was %s" % recommends[0].user
assert len(recommends) == 5, "the amount of recommendations was expected to be 5 but it was %s" % len(recommends)
test4 = str(recommends[1])
correct4 = "Rating(user=162, product=4074, rating=12.091128420713474)"
assert test4 == correct4, "the row was expected to be %s but it was %s" % (correct4, test4)

## Get Song Names

Get the song name and the artist for a given Rating object

Example

`[Rating(182412,218057,29.471691093542958),  
Rating(182412,206693,28.39630617887081),  
Rating(182412,230654,28.090021579109706),  
Rating(182412,200542,27.43900484648816),  
Rating(182412,254771,24.82362529344695)] ` 

You should create something like this  

`[["My Business","Guy"],  
["The Man With The Bag","Floyd/Animal/Zoot"],  
["Challenger","Growing"],  
["Una Ragazza In Due", "I Giganti"],
["That Is Why", "Say Anything"]]`  

First the song name and then the name of the band

You should start by converting the unique_tracks data into a dataframe. Columns are seperated by `<SEP>`. The correct schema is:

-> `track_id`: String  
-> `song_id`: String  
-> `artist`: String  
-> `title`: String

Next you should filter the `converted` dataframe based on if the `song_indexed` value is found in the Rating object array. Then you should Join the two dataframes and select the "title" and "artist" values. You will want to exclude duplicates. Finally, convert the dataframe into array (e.g convert it to rdd and use the collect() method).


param `converted` Dataframe created using `convert()` method  
param `ar` Array of Rating object produced using `recommendSongs()`  
param `path` path to unique track names file  
`return` corresponding song + artist names inside array, e.g., [['Our Song', 'Taylor Swift'],
 ['Boom (2006 Remastered Album Version)', 'P.O.D.']]


In [ ]:
def getSongNames(converted, ar, path):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
songNames = getSongNames(converted, recommends, sampleTracksPath)
songNames

In [ ]:
'''getSongNames test'''
assert len(songNames) == 5, "the amount of song names was expected to be 5 but it was %s" % len(songNames)
assert type(songNames[0]) == list, "the type of a songNames element was expected to be list but it was %s" % type(songNames[0])
test5 = songNames[3]
correct5 = ['Awakenings', 'Symphony X']
assert test5 == correct5, "the row was expected to be %s but it was %s" % (correct5, test5)

## Recommend

Recommend returns 5 song recommendations for a given user_id. Output should consists of Arrays that are size of 2. First element of the array is song name and the second artist name. You will basically just have to insert the parameters into the methods you have created. Remember the type of parameter that `recommendSongs()` accepts.

Example return

`[["My Business","Guy"],  
["The Man With The Bag","Floyd/Animal/Zoot"],  
["Challenger","Growing"],  
["Una Ragazza In Due", "I Giganti"],
["That Is Why", "Say Anything"]]`   

param `path` path to user data  
param `userId` user_id (String) that can be found from user dataset  
param `tracksPath` path to unique song names dataset   
param `seed` used for testing, put it into the `trainsALS()` method   
`return` corresponding song + artist names inside array

In [ ]:
def recommend(path, userId, tracksPath, seed):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
recom = recommend(sampleUsersPath, "b80344d063b5ccb3212f76538f3d9e43d87dca9e" ,sampleTracksPath, rSeed)
recom

In [ ]:
'''recommend test'''
assert len(recom) == 5, "the amount of recommendations was expected to be 5 but it was %s" % len(recom)
assert type(recom[0]) == list, "the type of a 'recommend' element was expected to be list but it was %s" % type(recom[0])
#test if the same user and seed returns the same as songNames
assert recom == songNames, "the song names were expected to be %s but they were %s" % (songNames, recom)

In [ ]:
spark.catalog.clearCache()
sc.stop()
spark.stop()